<a href="https://colab.research.google.com/github/hkbu-kennycheng/comp3925/blob/main/lab7_interactive_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# lab7 interactive visualization

Mounting Google Drive in Colab

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!apt update
!apt install chromium-browser chromium-chromedriver fonts-noto fonts-noto-cjk
!pip install pyppeteer nest_asyncio pandas numpy tqdm seaborn requests
# for local runtime with anaconda, please comment out above code and use following conda install intead
# !pip install pyppeteer nest_asyncio pandas numpy tqdm seaborn

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [1]:
import asyncio
import nest_asyncio
from pyppeteer import launch
import pandas as pd

nest_asyncio.apply()

In [3]:
def runAndReturn(func):
  return asyncio.get_event_loop().run_until_complete(asyncio.gather(func))[0]

# Data collection and processing

## HKTVMall

In [2]:
from time import time, sleep
import requests

def getLeafCategories():
  response = requests.get(f'https://apps.hktvmall.com/cate_map/eese/cate_code_convert.json?_={time()*1000}')
  rawdata = response.json()
  return [m['hktvmall_original_code'] for m in rawdata['cate_codes_map'] if not m['hktvmall_original_code'][-3:] == '000']


In [3]:
async def getCategoryTotalPages(catid):
  browser = await launch({'executablePath':'/usr/bin/chromium-browser', 'headless': True, 'args':['--no-sandbox']})
  # browser = await launch({'headless' :False})
  page = await browser.newPage() # open a new browser window
  await page.setUserAgent('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36')
  await page.setViewport({'width':1280, 'height':1024})
  await page.goto(f'https://www.hktvmall.com/hktv/en/search_a?category={catid}', { 'waitUntil':'networkidle0' })
  totalText = await page.Jeval('.paginationMenu-content span', 'node => node.textContent')
  await browser.close()
  return totalText.lstrip('/').rstrip('Pages').strip()

In [4]:
async def gethktvmall(catid):
  data = []
  total = int(await getCategoryTotalPages(catid))
  browser = await launch({'executablePath':'/usr/bin/chromium-browser', 'headless': True, 'args':['--no-sandbox']})
  # browser = await launch({'headless' :False})
  page = await browser.newPage() # open a new browser window
  await page.setUserAgent('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36')
  await page.setViewport({'width':1280, 'height':1024})
  for i in range(total):
    await page.goto(f'https://www.hktvmall.com/hktv/en/search_a?category={catid}&page={i}', { 'waitUntil':'networkidle0' })
    data += [{
      'name': ''.join(await e.JJeval('.brand-product-name','nodes => Array.from(nodes).map(node => node.textContent)')),
      'packing': ''.join(await e.JJeval('.packing-spec','nodes => Array.from(nodes).map(node => node.textContent)')),
      'sales': ''.join(await e.JJeval('.salesNumber-container','nodes => Array.from(nodes).map(node => node.textContent)')),
      'reviews': ''.join(await e.JJeval('.review-number','nodes => Array.from(nodes).map(node => node.textContent)')),
      'price': ''.join(await e.JJeval('.price','nodes => Array.from(nodes).map(node => node.textContent)')),
      'store_name': ''.join(await e.JJeval('.store-name-label','nodes => Array.from(nodes).map(node => node.textContent)')),
    } for e in await page.JJ('.product-brief-wrapper')]
  await browser.close()
  return pd.DataFrame(data)

In [6]:
runAndReturn(gethktvmall('AA11031000001'))

,name,packing,sales,reviews,price,store_name
0,MARUHA NICHIRO - Frozen Japanese Mentaiko with...,300G,"5,000+ Sold",(101),$ 115.00,Leader Marine
1,MARUHA NICHIRO - Frozen Japanese Karashi Menta...,300G,"6,000+ Sold",(167),$ 135.00,Leader Marine
2,MARUHA NICHIRO - Frozen JP Omelette x Karashi ...,ard 800g,800+ Sold,(19),$ 133.00,Maruha Nichiro Japanese Food Flagship Store
3,MARUHA NICHIRO - Frozen Japanese French Fry (M...,1KG,"7,000+ Sold",(155),$ 45.00,Leader Marine
4,CoolFood - Minji Angus Beef,400g,"20,000+ Sold",(258),$ 39.90,CoolFood
...,...,...,...,...,...,...
428,Batata Greens - 2Packs - Vegan Sweet and Sour ...,,,,$ 85.00,Happy Moment 丨開心生活-專門店
429,Batata Greens - 2Packs - Vegan Black Pepper Ch...,,,,$ 85.00,JK Organic 有機店｜天然健康食品用品生活專門店
430,Batata Greens - 2Packs - Vegan Sichuan Pepper ...,,,,$ 85.00,JK Organic 有機店｜天然健康食品用品生活專門店
431,Batata Greens - 2Packs - Vegan Algae Chinese P...,,,,$ 108.00,Happy Moment 丨開心生活-專門店


### Data collection

In [7]:
from tqdm import tqdm

df = pd.DataFrame()

for id in tqdm(getLeafCategories()):
  catdf = runAndReturn(gethktvmall(id))
  catdf.to_csv(f'/content/drive/MyDrive/{id}.csv')
  df = df.append(catdf)

  0%|          | 0/944 [00:00<?, ?it/s]

### Loading dataset from csv

In [5]:
!curl https://gist.githubusercontent.com/hkbu-kennycheng/d105b098eac03e2b40e477f9a49c62c7/raw/934d29f73165de57951083540d8b154a21d0a95f/AA11030500001.csv > AA11030500001.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/c0fe90b71521175bd03408487868df3d/raw/9d3481c3ec839bb4815a6cf3aa475fde3dc10591/AA11031000001.csv > AA11031000001.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/345cf3ca706b86891669e0881c21d851/raw/cef35e73b562e23da1d18dfd7161625eca801d02/AA11031500001.csv > AA11031500001.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/3446c57a872fc8ea61dba1361f8a2b7a/raw/d8e4da2e0c4862afd4ce253060c526fe4fec8592/AA11032000001.csv > AA11032000001.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/c196c678e9e73a60aeab74b348c81ea2/raw/471da27a11b380e44e22b3d341d0386cd3071d94/AA11032500001.csv > AA11032500001.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/e06364cea5ba0e3a4a5b589e44f5d423/raw/ec34429f5d913590b7ad205d925eec9df59e87fb/AA11033000001.csv > AA11033000001.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/59055c8db519c0505ed30f6c9f067919/raw/e83fbe517e8aba226f066ab268b58d06373604be/AA11033500001.csv > AA11033500001.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/920a659c68d0d30e360781fb4e02ee5f/raw/4a0159d903e9789698a40a07979b60ff89ea8ba6/AA11034000001.csv > AA11034000001.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/04a3ca0a0213513dbf9ed6475ccf76be/raw/5fdaf912513c1664b606813156d539980084ca14/AA11034500001.csv > AA11034500001.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/bf5eab5b219f27c9e0c57cc0a1cffa9e/raw/68b6708b3b38dbe385c47abb8c6cfd990a8854a5/AA11035000001.csv > AA11035000001.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/fb6e754e9e7c4674a012b05c2729637c/raw/fc2e23c7b37ce5e549e46581d3507828011d874f/AA11035500001.csv > AA11035500001.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/43a170c555d65383f2db4f7464821974/raw/222ae4102b08c755f75d1ec1a9795e9968d9dea7/AA11036000001.csv > AA11036000001.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/7e54fb29406b9a538d8eff62f6f23eec/raw/ae79c157075f12b542764e7f28ba4112750070a7/AA11037000001.csv > AA11037000001.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/b2e417657c30478274b0e81dc47ac9d9/raw/b818fd76a496ea93f0de7b37258574d1db38001e/AA11037500001.csv > AA11037500001.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/f6f6a34ff48094dde59ce5fe9f19de1d/raw/ee53cbcc20b203d955baff98c8bfafa315ee54a7/AA11038000001.csv > AA11038000001.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/215f8392e42e2a377467d1479bc54ef4/raw/cf23e855eea1c8cd3d1aa5c574964f1c872d4b19/AA11038500001.csv > AA11038500001.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/66f23d91a5dd41a66622fd6f6e564e31/raw/6be9c92db049d5b2452ee80cba0591597377a104/AA11039000001.csv > AA11039000001.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/97f55d88fdcf5d3fa64a417478562c83/raw/578efd4e169041f00b73ba175432c42b206794ad/AA11039500001.csv > AA11039500001.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/447ab8a7aa47fd7bc1b98751ac94504c/raw/6e2b32fac490ab732c6a700409245a10aa39d7c1/AA11081000001.csv > AA11081000001.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/01522ce70891f5f5105cd99c9515b799/raw/46c1ce974d5f03f9687ed1ed062e0045f2b1b756/AA11083000001.csv > AA11083000001.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/145af131c48f88a3974cca9113b163a7/raw/2521352189450e379a24792e8decf62565f8a833/AA11112600001.csv > AA11112600001.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/27b5b4a6155496dab3afc1e1b988cb20/raw/ab3d7622b9ef4c2d5ecacea7f898b1d0f3e8f6e2/AA11113200001.csv > AA11113200001.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2021  100  2021    0     0  12029      0 --:--:-- --:--:-- --:--:-- 12029
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 51243  100 51243    0     0   307k      0 --:--:-- --:--:-- --:--:--  307k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 77144  100 77144    0     0   492k      0 --:--:-- --:--:-- --:--:--  492k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  124k  100  124k    0     0   815k      0 --:--:-- --:--:-- --:--:--  815k
  % Total    % Received % Xferd  Average Speed   Tim

In [7]:
idslist = [
  'AA11030500001',
  'AA11031000001',
  'AA11031500001',
  'AA11032000001',
  'AA11032500001',
  'AA11033000001',
  'AA11033500001',
  'AA11034000001',
  'AA11034500001',
  'AA11035000001',
  'AA11035500001',
  'AA11036000001',
  'AA11037000001',
  'AA11037500001',
  'AA11038000001',
  'AA11038500001',
  'AA11039000001',
  'AA11039500001',
  'AA11081000001',
  'AA11083000001',
  'AA11112600001',
  'AA11113200001'
]

hktvmalldf = pd.DataFrame()
for id in idslist:
  hktvmalldf = hktvmalldf.append(pd.read_csv(f'/content/drive/MyDrive/{id}.csv'))

hktvmalldf

,Unnamed: 0,name,packing,sales,reviews,price,store_name
0,0,Chupa Chups - Direct from France - CHUPA CHUPS...,NaN,200+ Sold,(2),$ 41.00,LAZUR GOURMET
1,1,L'Azur Gourmet - PAYSANS D'ICI - Organic Apple...,NaN,60+ Sold,NaN,$ 79.00,LAZUR GOURMET
2,2,L'Azur Gourmet - CLUB DES SOMMELIERS - Banyuls...,NaN,10+ Sold,NaN,$ 133.00,LAZUR GOURMET
3,3,Cradle Fresh - CLUB DES SOMMELIERS - Maury Ros...,NaN,NaN,NaN,$ 136.00,LAZUR GOURMET
4,4,Grace Cup - D197 Musang King Frozen Durian Pul...,200 g,100+ Sold,(3),$ 198.00,Mandarin Kitchen
...,...,...,...,...,...,...,...
185,185,PAK FOOK - Freash Beancurd Dessert (6 packs) ...,229gx6,200+ Sold,(8),$ 35.40,Milk Factory
186,186,PAK FOOK - PAK FOOK Nigari Super Hard BeanC 25...,250gx2,NaN,NaN,$ 27.00,Milk Factory
187,187,Chef's Workshop - (Frozen) Edamame (Wasabi) (4...,400g,10+ Sold,(1),$ 32.00,Chef's Workshop (HK) Limited
188,188,Fresh To Go - Thai Vermicelli,NaN,NaN,NaN,$ 36.00,FRESH TO GO 餸菜包


## ParkNShop

### Loading data from csv

In [8]:
!curl https://gist.githubusercontent.com/hkbu-kennycheng/6e20e5965916e85008701f16e8e41a29/raw/199efc8496594a3539ed26f52ddfb774d8e5c0c1/pnk0.csv > pnk0.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/36e05f717c4720fcecc45ddcf613639b/raw/74bebec8fe652f91b360fdc9bc26ba84d6209f8c/pnk1.csv > pnk1.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/776469f16450995ac4f4279f9b382949/raw/ef4b582abb61ddda09788d997c2c2a2b32e6462e/pnk2.csv > pnk2.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/46c0cbc4cfc2dc55c5c6b62125fa3dd9/raw/457719baf2555bb463ac878ae586d8062f9f5bd5/pnk3.csv > pnk3.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/4db0f47c2d995524e8c06604db25e160/raw/1303cf57d9a4df48c2b750e509ca20bd290fdd3a/pnk4.csv > pnk4.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/7f0caf13c2031862e88cec5f0d93e666/raw/8bd19b6b3c3f961522b6e5d5c13a54060e4206a5/pnk5.csv > pnk5.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/852fd856ee15d741a4cc9c7bab6053e3/raw/3533fd12dda720b2ab3ba857b04375560f72bc8e/pnk6.csv > pnk6.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/d6135994e19e121e6494c42e6bb6f526/raw/2daaa9b13f0ad7209b13212ae346d23918b439a8/pnk7.csv > pnk7.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/8d82f5cd26f2b8b2272ee7c031ca34e7/raw/625678d8266631b57752a13b25cc0307f3e7d187/pnk8.csv > pnk8.csv
!curl https://gist.githubusercontent.com/hkbu-kennycheng/b32b8000273e8dab592987e19e5acaf4/raw/d1f4c5bd31be4191543a59465ecdbedd2c3d472f/pnk9.csv > pnk9.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1022k  100 1022k    0     0  3060k      0 --:--:-- --:--:-- --:--:-- 3051k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1039k  100 1039k    0     0  4210k      0 --:--:-- --:--:-- --:--:-- 4210k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1030k  100 1030k    0     0  4058k      0 --:--:-- --:--:-- --:--:-- 4058k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1035k  100 1035k    0     0  3982k      0 --:--:-- --:--:-- --:--:-- 3967k
  % Total    % Received % Xferd  Average Speed   Tim

In [9]:
import pandas as pd

df = pd.read_csv('pnk0.csv', index_col=0)
for i in range(1,10):
  df = df.append(pd.read_csv(f'pnk{i}.csv', index_col=0), ignore_index=True)

df

,name,id,url,pic,price,specialoffer,brand,category,variant,variantname,topseller,fairtrade,organic,eshoponly,healthyfat,crustacean
0,Alkaline Water 9.0 (random Pick),499585,https://www.parknshop.com/en/alkaline-water-9-...,https://www.parknshop.com/medias/sys_master/fr...,9.0,NaN,WATSONS,Water,499585,420ML,False,False,False,False,False,False
1,Alkaline Water 9.0 (case) (420ml X 24 Bottles),378829,https://www.parknshop.com/en/alkaline-water-9-...,https://www.parknshop.com/medias/sys_master/fr...,168.0,NaN,WATSONS,"Water,Bottled Water",378829,24X420ML,False,False,False,False,False,False
2,L Type Interdental Brush 0.6mm - 6pcs,805629,https://www.parknshop.com/en/l-type-interdenta...,https://www.parknshop.com/medias/sys_master/fr...,13.9,\n\t\t\t\t\t\t$20.8 / 2pcs\n\t\t\t\t\t\t,WATSONS-,Dental Accessories & Denture,805629,6S,False,False,False,False,False,False
3,L Type Interdental Brush 0.8mm 6pcs,805630,https://www.parknshop.com/en/l-type-interdenta...,https://www.parknshop.com/medias/sys_master/fr...,13.9,\n\t\t\t\t\t\t$20.8 / 2pcs\n\t\t\t\t\t\t,WATSONS-,Dental Accessories & Denture,805630,6S,True,False,False,False,False,False
4,I Type Interdental Brush 0.7mm 5pcs,805627,https://www.parknshop.com/en/i-type-interdenta...,https://www.parknshop.com/medias/sys_master/fr...,10.0,\n\t\t\t\t\t\t$15.0 / 2pcs\n\t\t\t\t\t\t,WATSONS-,Dental Accessories & Denture,805627,5S,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28995,Digestive Oat,116926,https://www.parknshop.com/en/digestive-oat/p/B...,https://www.parknshop.com/medias/sys_master/fr...,22.9,\n\t\t\t\t\t\tMONEYBACK MEMBER OFFER\n\t\t\t\t...,MCVITIE'S,Other Sweet Biscuits,116926,300G,True,False,False,False,False,False
28996,Pocky Strawberry (hk Ver.),117573,https://www.parknshop.com/en/pocky-tsubu-straw...,https://www.parknshop.com/medias/sys_master/fr...,16.5,\n\t\t\t\t\t\tMONEYBACK MEMBER OFFER\n\t\t\t\t...,GLICO,Biscuit Sticks & Pretzels,117573,51G,True,False,False,False,False,False
28997,Veggie Wash,118309,https://www.parknshop.com/en/fruit-and-vegetab...,https://www.parknshop.com/medias/sys_master/fr...,60.9,\n\t\t\t\t\t\tMONEYBACK MEMBER OFFER\n\t\t\t\t...,CITRUS MAGIC,Kitchen Cleaners,118309,16OZ,False,False,False,False,False,False
28998,Longan In Syrup,125948,https://www.parknshop.com/en/longan-in-syrup/p...,https://www.parknshop.com/medias/sys_master/fr...,26.9,\n\t\t\t\t\t\tMONEYBACK MEMBER OFFER\n\t\t\t\t...,TWIN ELEPHANTS,Fruits,125948,565G,True,False,False,False,False,False


### Data cleaning

Cleaning empty space characters with strip for object columns

In [7]:
df['name'] = df['name'].str.strip()
df['url'] = df['url'].str.strip()
df['pic'] = df['pic'].str.strip()
df['specialoffer'] = df['specialoffer'].str.strip()
df['brand'] = df['brand'].str.strip()
df['category'] = df['category'].str.strip()
df['variantname'] = df['variantname'].str.strip()

df

,name,id,url,pic,price,specialoffer,brand,category,variant,variantname,topseller,fairtrade,organic,eshoponly,healthyfat,crustacean
0,Alkaline Water 9.0 (random Pick),499585,https://www.parknshop.com/en/alkaline-water-9-...,https://www.parknshop.com/medias/sys_master/fr...,9.0,NaN,WATSONS,Water,499585,420ML,False,False,False,False,False,False
1,Alkaline Water 9.0 (case) (420ml X 24 Bottles),378829,https://www.parknshop.com/en/alkaline-water-9-...,https://www.parknshop.com/medias/sys_master/fr...,168.0,NaN,WATSONS,"Water,Bottled Water",378829,24X420ML,False,False,False,False,False,False
2,L Type Interdental Brush 0.6mm - 6pcs,805629,https://www.parknshop.com/en/l-type-interdenta...,https://www.parknshop.com/medias/sys_master/fr...,13.9,$20.8 / 2pcs,WATSONS-,Dental Accessories & Denture,805629,6S,False,False,False,False,False,False
3,L Type Interdental Brush 0.8mm 6pcs,805630,https://www.parknshop.com/en/l-type-interdenta...,https://www.parknshop.com/medias/sys_master/fr...,13.9,$20.8 / 2pcs,WATSONS-,Dental Accessories & Denture,805630,6S,True,False,False,False,False,False
4,I Type Interdental Brush 0.7mm 5pcs,805627,https://www.parknshop.com/en/i-type-interdenta...,https://www.parknshop.com/medias/sys_master/fr...,10.0,$15.0 / 2pcs,WATSONS-,Dental Accessories & Denture,805627,5S,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28995,Digestive Oat,116926,https://www.parknshop.com/en/digestive-oat/p/B...,https://www.parknshop.com/medias/sys_master/fr...,22.9,MONEYBACK MEMBER OFFER,MCVITIE'S,Other Sweet Biscuits,116926,300G,True,False,False,False,False,False
28996,Pocky Strawberry (hk Ver.),117573,https://www.parknshop.com/en/pocky-tsubu-straw...,https://www.parknshop.com/medias/sys_master/fr...,16.5,MONEYBACK MEMBER OFFER,GLICO,Biscuit Sticks & Pretzels,117573,51G,True,False,False,False,False,False
28997,Veggie Wash,118309,https://www.parknshop.com/en/fruit-and-vegetab...,https://www.parknshop.com/medias/sys_master/fr...,60.9,MONEYBACK MEMBER OFFER,CITRUS MAGIC,Kitchen Cleaners,118309,16OZ,False,False,False,False,False,False
28998,Longan In Syrup,125948,https://www.parknshop.com/en/longan-in-syrup/p...,https://www.parknshop.com/medias/sys_master/fr...,26.9,MONEYBACK MEMBER OFFER,TWIN ELEPHANTS,Fruits,125948,565G,True,False,False,False,False,False


### Drop duplicates with id

In [13]:
len(df['id'].unique())

6727

In [14]:
df.drop_duplicates('id' ,inplace=True)

In [15]:
df

,name,id,url,pic,price,specialoffer,brand,category,variant,variantname,topseller,fairtrade,organic,eshoponly,healthyfat,crustacean
0,Alkaline Water 9.0 (random Pick),499585,https://www.parknshop.com/en/alkaline-water-9-...,https://www.parknshop.com/medias/sys_master/fr...,9.0,NaN,WATSONS,Water,499585,420ML,False,False,False,False,False,False
1,Alkaline Water 9.0 (case) (420ml X 24 Bottles),378829,https://www.parknshop.com/en/alkaline-water-9-...,https://www.parknshop.com/medias/sys_master/fr...,168.0,NaN,WATSONS,"Water,Bottled Water",378829,24X420ML,False,False,False,False,False,False
2,L Type Interdental Brush 0.6mm - 6pcs,805629,https://www.parknshop.com/en/l-type-interdenta...,https://www.parknshop.com/medias/sys_master/fr...,13.9,$20.8 / 2pcs,WATSONS,Dental Accessories & Denture,805629,6S,False,False,False,False,False,False
3,L Type Interdental Brush 0.8mm 6pcs,805630,https://www.parknshop.com/en/l-type-interdenta...,https://www.parknshop.com/medias/sys_master/fr...,13.9,$20.8 / 2pcs,WATSONS,Dental Accessories & Denture,805630,6S,True,False,False,False,False,False
4,I Type Interdental Brush 0.7mm 5pcs,805627,https://www.parknshop.com/en/i-type-interdenta...,https://www.parknshop.com/medias/sys_master/fr...,10.0,$15.0 / 2pcs,WATSONS,Dental Accessories & Denture,805627,5S,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26909,Kitchen Towel 9inch,365593,https://www.parknshop.com/en/kitchen-towel-9in...,https://www.parknshop.com/medias/sys_master/fr...,25.9,MONEYBACK MEMBER OFFER,BEST BUY,NaN,365593,6S,True,False,False,False,False,False
27000,Chauzhou Style Fish Balls,51761,https://www.parknshop.com/en/chauzhou-style-fi...,https://www.parknshop.com/medias/sys_master/fr...,8.4,MONEYBACK MEMBER OFFER,JIU LONG CHENG ZHAI,Seafood Balls,51761,170G/PKT,True,False,False,False,False,False
27001,Alkaline Ion Water,483353,https://www.parknshop.com/en/alkaline-ion-wate...,https://www.parknshop.com/medias/sys_master/fr...,19.9,MONEYBACK MEMBER OFFER,KIRIN,Water,483353,2L,False,False,False,False,False,False
27002,Mushroom Pork Ball,55251,https://www.parknshop.com/en/mushroom-pork-bal...,https://www.parknshop.com/medias/sys_master/fr...,13.9,MONEYBACK MEMBER OFFER,JIU LONG CHENG ZHAI,Meat Balls,55251,170G,True,False,False,False,False,False


### Grouping brand name

Check for dash in `df['brand']`

In [8]:
df[df['brand'].str.count('-') > 0]['brand'].describe()

count         647
unique         23
top       DE-MA-E
freq          213
Name: brand, dtype: object

replace dash with empty string in `df['brand']`

In [9]:
df['brand'] = df['brand'].str.replace('-','')
df['brand'].describe()

count      28600
unique      1468
top       SELECT
freq        2392
Name: brand, dtype: object

In [10]:
df

,name,id,url,pic,price,specialoffer,brand,category,variant,variantname,topseller,fairtrade,organic,eshoponly,healthyfat,crustacean
0,Alkaline Water 9.0 (random Pick),499585,https://www.parknshop.com/en/alkaline-water-9-...,https://www.parknshop.com/medias/sys_master/fr...,9.0,NaN,WATSONS,Water,499585,420ML,False,False,False,False,False,False
1,Alkaline Water 9.0 (case) (420ml X 24 Bottles),378829,https://www.parknshop.com/en/alkaline-water-9-...,https://www.parknshop.com/medias/sys_master/fr...,168.0,NaN,WATSONS,"Water,Bottled Water",378829,24X420ML,False,False,False,False,False,False
2,L Type Interdental Brush 0.6mm - 6pcs,805629,https://www.parknshop.com/en/l-type-interdenta...,https://www.parknshop.com/medias/sys_master/fr...,13.9,$20.8 / 2pcs,WATSONS,Dental Accessories & Denture,805629,6S,False,False,False,False,False,False
3,L Type Interdental Brush 0.8mm 6pcs,805630,https://www.parknshop.com/en/l-type-interdenta...,https://www.parknshop.com/medias/sys_master/fr...,13.9,$20.8 / 2pcs,WATSONS,Dental Accessories & Denture,805630,6S,True,False,False,False,False,False
4,I Type Interdental Brush 0.7mm 5pcs,805627,https://www.parknshop.com/en/i-type-interdenta...,https://www.parknshop.com/medias/sys_master/fr...,10.0,$15.0 / 2pcs,WATSONS,Dental Accessories & Denture,805627,5S,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28995,Digestive Oat,116926,https://www.parknshop.com/en/digestive-oat/p/B...,https://www.parknshop.com/medias/sys_master/fr...,22.9,MONEYBACK MEMBER OFFER,MCVITIE'S,Other Sweet Biscuits,116926,300G,True,False,False,False,False,False
28996,Pocky Strawberry (hk Ver.),117573,https://www.parknshop.com/en/pocky-tsubu-straw...,https://www.parknshop.com/medias/sys_master/fr...,16.5,MONEYBACK MEMBER OFFER,GLICO,Biscuit Sticks & Pretzels,117573,51G,True,False,False,False,False,False
28997,Veggie Wash,118309,https://www.parknshop.com/en/fruit-and-vegetab...,https://www.parknshop.com/medias/sys_master/fr...,60.9,MONEYBACK MEMBER OFFER,CITRUS MAGIC,Kitchen Cleaners,118309,16OZ,False,False,False,False,False,False
28998,Longan In Syrup,125948,https://www.parknshop.com/en/longan-in-syrup/p...,https://www.parknshop.com/medias/sys_master/fr...,26.9,MONEYBACK MEMBER OFFER,TWIN ELEPHANTS,Fruits,125948,565G,True,False,False,False,False,False


# Interactive visualization

In [ ]:
!pip install altair vega_datasets


https://altair-viz.github.io

![](https://url2img-web.herokuapp.com/aHR0cHM6Ly9hbHRhaXItdml6LmdpdGh1Yi5pby8=)


https://altair-viz.github.io/gallery/index.html#interactive-charts
![](https://url2img-web.herokuapp.com/aHR0cHM6Ly9hbHRhaXItdml6LmdpdGh1Yi5pby9nYWxsZXJ5L2luZGV4Lmh0bWwjaW50ZXJhY3RpdmUtY2hhcnRz)